In [1]:
import os
import numpy as np
import pandas as pd

from shutil import copyfile

In [2]:
image_root = 'C:/Joconde/joconde'
top_category =  'domaine par support de conservation'#'genre de la représentation/genre iconographique' #'arme de main'
class_root = os.path.join(image_root, top_category)
group_by = 'category' # can be either 'category' or 'parent'

## Read Stored Data From The Query Script

In [3]:
%store -r image_set_df

print(image_set_df.shape)
image_set_df.head()

(242430, 4)


,category,parent,imagePath,ref
0,art mobilier,domaine par support de conservation,/arc/0006/m500145_0000711_p.jpg,50010004390
1,art mobilier,domaine par support de conservation,/arc/0010/m500145_0016928_p.jpg,50010005721
2,art mobilier,domaine par support de conservation,/arc/0007/m500145_0002121_p.jpg,50010004447
3,art mobilier,domaine par support de conservation,/arc/0009/m500145_0007336_p.jpg,50010008271
4,art mobilier,domaine par support de conservation,/0055/m500145_0013990_p.jpg,50010008370


In [4]:
if('width' in image_set_df.columns):
    image_set_df.drop(['width', 'height', 'usage'],axis=1 , inplace=True)
else:
    print ('nothing to do')

nothing to do


## Augment Data

#### Read image sizes from flat file (faster)

In [5]:
csv_file_name = '..\Joconde\main_image_size.csv'
image_size_df = pd.read_csv(csv_file_name)

print(image_size_df.shape)
image_size_df.head()

(298511, 5)


,Unnamed: 0,ref,imagePath,width,height
0,0,AG021125,/0059/m079057_0001758_p.jpg,215,481.0
1,1,AG021121,/0059/m079057_0001663_p.jpg,183,512.0
2,2,AG019906,/0059/m079057_0002134_p.jpg,170,488.0
3,3,AG019903,/0059/m079057_0002122_p.jpg,204,502.0
4,4,AG019754,/0059/m079057_0001895_p.jpg,197,512.0


In [6]:
pd.merge(image_set_df,
        image_size_df[['ref', 'width', 'height']],
        on='ref',
        how='left').shape

(242430, 6)

In [7]:
try:
    image_size_df
except NameError:
    image_set_df['width'] = 0
    image_set_df['height'] = 0
    image_set_df['usage'] = 'remain'
else:
    image_set_df = pd.merge(image_set_df,
                            image_size_df[['ref', 'width', 'height']],
                            on='ref',
                            how='left')
    image_set_df.fillna(0 , inplace=True)
    image_set_df['usage'] = 'remain'

error_count = 0
new_count = 0

for i, row in image_set_df[image_set_df.width == 0].iterrows():

    if os.path.isfile(image_root + row.imagePath) :
        try:

            image = Image.open(image_root + row.imagePath)

            # get image size
            #images_df.loc[i, ['exists']] = True
            image_set_df.loc[i, ['width']] = image.size[0]
            image_set_df.loc[i, ['height']] = image.size[1]

            image.close()
            new_count += 1
        except:
            error_count += 1
    if i % 1000 == 0:
        print (i, end=', ')

print()            
print('updated size %d; detected %d errors' %  (new_count, error_count))
            

0, 80000, 181000, 219000, 223000, 238000, 
updated size 0; detected 0 errors


In [8]:
image_set_df.head()

,category,parent,imagePath,ref,width,height,usage
0,art mobilier,domaine par support de conservation,/arc/0006/m500145_0000711_p.jpg,50010004390,0,0.0,remain
1,art mobilier,domaine par support de conservation,/arc/0010/m500145_0016928_p.jpg,50010005721,0,0.0,remain
2,art mobilier,domaine par support de conservation,/arc/0007/m500145_0002121_p.jpg,50010004447,0,0.0,remain
3,art mobilier,domaine par support de conservation,/arc/0009/m500145_0007336_p.jpg,50010008271,0,0.0,remain
4,art mobilier,domaine par support de conservation,/0055/m500145_0013990_p.jpg,50010008370,640,480.0,remain


#### To IPython database

In [9]:
%store image_set_df

Stored 'image_set_df' (DataFrame)


## Filter Data


In [18]:
# filter out the records that have invalid image path
# filter out the images that are too far from being square

aspect_ratio_thershold = 2.0
large_category_threshold = 800

print(image_set_df.shape , ' total')
print(image_set_df[image_set_df.width > 0].shape, 'images exist')

filtered_df = image_set_df[(image_set_df.width > 0) &
                        (image_set_df.width/image_set_df.height <= aspect_ratio_thershold) &
                        (image_set_df.height/image_set_df.width <= aspect_ratio_thershold) ]

print(filtered_df.shape , 'required aspect ratio %.2f' % aspect_ratio_thershold)


# filter only large categoties
filtered_df = filtered_df.groupby(by=group_by).filter(lambda x: len(x) >= large_category_threshold) 
    

print(filtered_df.shape , 'in %d grouping categories that have more than %d images' %
                          ( len(filtered_df.groupby(by=group_by).size()), large_category_threshold))

# print top populous categories
print(image_set_df.groupby(by=group_by).size())
print(filtered_df.groupby(by=group_by).size())

filtered_df.head()

(242430, 7)  total
(236981, 7) images exist
(231084, 7) required aspect ratio 2.00
(230843, 7) in 8 grouping categories that have more than 800 images
category
art mobilier       906
dessin          155334
enluminure          69
estampe          24237
imprimé           1019
manuscrit          990
miniature         1820
peinture         31901
photographie     15492
plumasserie          4
sculpture        10658
dtype: int64
category
dessin          152930
estampe          23492
imprimé            921
manuscrit          977
miniature         1808
peinture         30420
photographie     10900
sculpture         9395
dtype: int64


,category,parent,imagePath,ref,width,height,usage
906,dessin,domaine par support de conservation,/0266/m503501_d0222117-000_p.jpg,50350222117,341,512.0,remain
907,dessin,domaine par support de conservation,/0155/m503501_d0122722-000_p.jpg,50350122722,341,512.0,remain
908,dessin,domaine par support de conservation,/0553/m033201_1170297-1_p.jpg,B3321170297,800,552.0,remain
909,dessin,domaine par support de conservation,/0285/m503501_d0003119-000_p.jpg,50350003119,341,512.0,remain
910,dessin,domaine par support de conservation,/0228/m503501_d0133235-000_p.jpg,50350133235,341,512.0,remain


In [19]:
## Remove some categories that can overlap 
classes_to_remove = ["manuscrit"] 
classes_to_keep   = []

filtered_df = filtered_df[~filtered_df[group_by].isin(classes_to_remove)]
#filtered_df = filtered_df[filtered_df[group_by].isin(classes_to_keep)]

print(filtered_df.shape , 'in %d grouped categories that have more than %d images' %
                          ( len(filtered_df.groupby(by=group_by).size()), large_category_threshold))
print(filtered_df.groupby(by=group_by).size())

filtered_df.head()

(229866, 7) in 7 grouped categories that have more than 800 images
category
dessin          152930
estampe          23492
imprimé            921
miniature         1808
peinture         30420
photographie     10900
sculpture         9395
dtype: int64


,category,parent,imagePath,ref,width,height,usage
906,dessin,domaine par support de conservation,/0266/m503501_d0222117-000_p.jpg,50350222117,341,512.0,remain
907,dessin,domaine par support de conservation,/0155/m503501_d0122722-000_p.jpg,50350122722,341,512.0,remain
908,dessin,domaine par support de conservation,/0553/m033201_1170297-1_p.jpg,B3321170297,800,552.0,remain
909,dessin,domaine par support de conservation,/0285/m503501_d0003119-000_p.jpg,50350003119,341,512.0,remain
910,dessin,domaine par support de conservation,/0228/m503501_d0133235-000_p.jpg,50350133235,341,512.0,remain


In [ ]:
c = pd.crosstab(index=filtered_df.ref, 
                              columns="ci")
c[c.ci > 1]


In [15]:
# filter out images with more than 1 subject from the category
filtered_df = filtered_df.groupby(by='ref').filter(lambda x: len(x) == 1) 

filtered_df = filtered_df.groupby(by=group_by).filter(lambda x: len(x) >= 300) 

print(filtered_df.shape , 'single class images' )
print(filtered_df.groupby(by=group_by).size())

(229448, 7) single class images
category
dessin          152889
estampe          23392
imprimé            779
miniature         1762
peinture         30395
photographie     10837
sculpture         9394
dtype: int64


## Straified Split

In [20]:
def train_validate_test_split(df, train_percent=.8, val_percent=.1, test_percent= 0.1, seed=None, max_size=0):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df)
    
    if (max_size > 0 and m > max_size):
        m = max_size
    
    #train_end = int(train_percent * m)
    test_end = int(test_percent * m)
    #val_end = int(val_percent * m) + train_end
    val_end = int(val_percent * m) + test_end
    
    
    
    #train_idx = perm[:train_end]
    test_idx = perm[:test_end]
    #val_idx   = perm[train_end:val_end]
    val_idx   = perm[test_end:val_end]
    #test_idx  = perm[val_end: m]
    train_idx  = perm[val_end: m]
    
   
    return train_idx, val_idx, test_idx

In [21]:
cat_list = filtered_df.groupby(by=group_by).size()

min_strata_size = cat_list.min()

dataset_split = {'train': 0.8,
                 'val':   0.1,
                 'test':  0.1}

#cat_df = filtered_df.groupby(by='category')
#cat_list = cat_df.size()
#cat_list = cat_list[cat_list > min_strata_size]

filtered_df['usage'] = 'remain'

for category in cat_list.keys():
    train_idx, val_idx, test_idx = train_validate_test_split(filtered_df[filtered_df[group_by] == category],
                                                             train_percent = dataset_split['train'],
                                                             val_percent = dataset_split['val'],
                                                             test_percent = dataset_split['test'],
                                                             max_size = min_strata_size)
    
    filtered_df.loc[train_idx , ['usage']] = 'train'
    filtered_df.loc[val_idx , ['usage']] = 'val'
    filtered_df.loc[test_idx , ['usage']] = 'test'


# Two way frequency table
pd.crosstab(index=filtered_df[group_by], 
                  columns=filtered_df['usage'])

usage,remain,test,train,val
category,,,,
dessin,152009,92,737,92
estampe,22571,92,737,92
imprimé,0,92,737,92
miniature,887,92,737,92
peinture,29499,92,737,92
photographie,9979,92,737,92
sculpture,8474,92,737,92


In [22]:
## prep the destination for split dataset
save_remaining = True

if not os.path.exists(class_root):
    os.makedirs(class_root)
    
for split_name in  dataset_split.keys():
    if dataset_split[split_name] > 0 and not os.path.exists(os.path.join(class_root, split_name)):
        os.makedirs(os.path.join(class_root, split_name))

if save_remaining and not os.path.exists(os.path.join(class_root, 'remain')):
    os.makedirs(os.path.join(class_root, 'remain'))

## Save Images for Classification

NOTE: image files are renamed to carry the notice reference forward for analysis

In [23]:
for i, row in filtered_df.iterrows():
    
    src_file_path = image_root + row.imagePath

    dest_dir_name = os.path.join (class_root, row.usage, row[group_by])
   
    fn, ext = os.path.splitext(os.path.basename(row.imagePath))
    
    dest_file_path = os.path.join(dest_dir_name, row.ref + ext) #rename file to notice reference
        
    if not os.path.exists(dest_dir_name):
        os.makedirs(dest_dir_name)

    copyfile(src_file_path, dest_file_path)
    
    if i % 1000 == 0:
        print (i, end=', ')
  
print (i)

1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 15000, 16000, 17000, 18000, 19000, 20000, 21000, 22000, 23000, 24000, 25000, 26000, 27000, 28000, 29000, 30000, 31000, 32000, 33000, 34000, 35000, 36000, 37000, 38000, 39000, 40000, 41000, 42000, 43000, 44000, 45000, 46000, 47000, 48000, 49000, 50000, 51000, 52000, 53000, 54000, 55000, 56000, 57000, 58000, 59000, 60000, 61000, 62000, 63000, 64000, 65000, 66000, 67000, 68000, 69000, 70000, 71000, 72000, 73000, 74000, 75000, 76000, 77000, 78000, 79000, 81000, 82000, 83000, 84000, 85000, 86000, 87000, 88000, 89000, 90000, 91000, 92000, 93000, 94000, 95000, 96000, 97000, 98000, 99000, 100000, 101000, 102000, 103000, 104000, 105000, 106000, 107000, 108000, 109000, 110000, 111000, 112000, 113000, 114000, 115000, 116000, 117000, 118000, 119000, 120000, 121000, 122000, 123000, 124000, 125000, 126000, 127000, 128000, 129000, 130000, 131000, 132000, 133000, 134000, 135000, 136000, 137000, 138000, 139000, 140000, 14

## Scrapbook

In [ ]:
from shutil import move

root_fld = 'C:/Joconde/joconde/domaine par support de conservation/remain'
for cls in os.listdir(root_fld):
    for fn in os.listdir(os.path.join(root_fld, cls)):
        ref = images_df[images_df.path.str.endswith(fn)].ref.tolist()
        if(len(ref) > 0):
            
            fn_path = os.path.join(root_fld, cls, fn)
            fn_new = fn_path.replace(os.path.splitext(fn)[0] , ref[0])
            move(fn_path, fn_new)
            print(fn_new)
            
        
 